In [ ]:
!pip install gensim

In [ ]:
!pip install nltk

In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import gensim
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import GradientBoostingClassifier

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
bully_df = pd.read_csv('./drive/MyDrive/datasets/cyberbullying_tweets.csv')

In [ ]:
bully_df.tail()

In [ ]:
bully_df = bully_df.rename(columns={'tweet_text':'tweet','cyberbullying_type':'type'})

In [ ]:
bully_df.head()

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lambda x: re.sub(r'@\w+', '', x).strip())

In [ ]:
bully_df.head()

In [ ]:
def cleantxt(txt):
    stpw = stopwords.words('english')

    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-zA-Z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(cleantxt)

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lambda x: remove_stopwords(x))

In [ ]:
bully_df.tail(10)

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    tokens = nltk.word_tokenize(sentence)
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in tokens]
    return ' '.join(lemmatized_words)

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lemmatize_sentence)

In [ ]:
bully_df.tail(10)

In [ ]:
bully_df.tail(10)

In [ ]:
from wordcloud import WordCloud
negative_words = ' '.join([text for text in bully_df['tweet'][bully_df['type'] != 'not_cyberbullying']])
wordcloud = WordCloud(width=800, height=500,
random_state=21, max_font_size=110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lambda x: x.split())
# bully_df['tweet'] = bully_df['tweet'].apply(lambda x: ([w for w in x if len(w)>=3]))
# bully_df['tweetcnn'] = bully_df['tweet'].apply(lambda x: ' '.)

In [ ]:
bully_df.tail(10)

In [ ]:
# gender religion age ethnicity other_cyberbullying
not_bully_df = bully_df[bully_df['type'] == 'not_cyberbullying']
gender_bully_df = bully_df[bully_df['type'] == 'gender']
religion_bully_df = bully_df[bully_df['type'] == 'religion']
age_bully_df = bully_df[bully_df['type'] == 'age']
ethnicity_bully_df = bully_df[bully_df['type'] == 'ethnicity']
other_bully_df = bully_df[bully_df['type'] == 'other_cyberbullying']
all_bully_df = bully_df[bully_df['type'] != 'not_cyberbullying']

print(not_bully_df.shape)
print(gender_bully_df.shape)
print(religion_bully_df.shape)
print(age_bully_df.shape)
print(ethnicity_bully_df.shape)
print(other_bully_df.shape)
print(all_bully_df.shape)

In [ ]:
not_bully_df.tail()

In [ ]:
sampled_not_bully_df = not_bully_df.sample(n=7000, random_state=42)
sampled_gender_bully_df = gender_bully_df.sample(n=1750, random_state=42)
sampled_age_bully_df = age_bully_df.sample(n=1750, random_state=42)
sampled_religion_bully_df = religion_bully_df.sample(n=1750, random_state=42)
sampled_ethnicity_bully_df = ethnicity_bully_df.sample(n=1750, random_state=42)
sampled_other_bully_df = other_bully_df.sample(n=1500, random_state=42)

sampled2_not_bully_df = not_bully_df.sample(n=3000, random_state=42)
sampled2_gender_bully_df = gender_bully_df.sample(n=600, random_state=42)
sampled2_age_bully_df = age_bully_df.sample(n=600, random_state=42)
sampled2_religion_bully_df = religion_bully_df.sample(n=600, random_state=42)
sampled2_ethnicity_bully_df = ethnicity_bully_df.sample(n=600, random_state=42)
sampled2_other_bully_df = other_bully_df.sample(n=600, random_state=42)

In [ ]:
sampled_gender_bully_df.shape

In [ ]:
sampled_gender_bully_df.tail()

In [ ]:
final_df = pd.concat([sampled_not_bully_df, sampled_gender_bully_df, sampled_age_bully_df, sampled_religion_bully_df, sampled_ethnicity_bully_df], ignore_index=True)
final_df['tweet'] = final_df['tweet'].apply(lambda x: ([w for w in x if len(w)>=3]))

In [ ]:
final2_df = pd.concat([sampled2_not_bully_df, sampled2_gender_bully_df, sampled2_age_bully_df, sampled2_religion_bully_df, sampled2_ethnicity_bully_df, sampled2_other_bully_df], ignore_index=True)
final2_df['tweet'] = final2_df['tweet'].apply(lambda x: ([w for w in x if len(w)>=3]))

In [ ]:
print(final_df['tweet'])

In [ ]:
final2_df.loc[final2_df['type']!='not_cyberbullying','type'] = 'yes'
final2_df.loc[final2_df['type']=='not_cyberbullying','type'] = 'no'

In [ ]:
# def allign(text):
#   while len(text)<5:
#     text.append('<oov>')
#   return text

# final_df['tweet'] = final_df['tweet'].apply(allign)
# final2_df['tweet'] = final2_df['tweet'].apply(allign)

In [ ]:
mapping = {'not_cyberbullying': 0, 'gender': 1, 'age': 1, 'ethnicity': 1, 'religion':1, 'other_cyberbullying':1}
mapping2 = {'yes': 1, 'no': 0}

In [ ]:
final_df.tail(20)

In [ ]:
final_df['type'] = final_df['type'].map(mapping)
final2_df['type'] = final2_df['type'].map(mapping2)

In [ ]:
final_df.tail()

In [ ]:
final_df.shape

In [ ]:
def get3gram(text):
  arr = []
  if len(text)<3:
    arr.append('-'.join(text))
  for i in range(2,len(text)):
    substr = text[i-2:i+1]
    st = '-'.join(substr)
    arr.append(st)
  return arr

final_df['tweet3gram'] = final_df['tweet'].apply(get3gram)
final2_df['tweet3gram'] = final2_df['tweet'].apply(get3gram)

In [ ]:
final_df.tail(20)

In [ ]:
final_df.shape

### 1 gram multiple type

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(final_df['tweet'])
corpus = [dictionary.doc2bow(text) for text in final_df['tweet']]
tfidf = gensim.models.TfidfModel(corpus)

In [ ]:
num_docs = len(corpus)
num_terms = len(dictionary)

tfidf_matrix = np.zeros((num_docs, num_terms))

for doc_index, doc in enumerate(tfidf[corpus]):
    for word_id, score in doc:
        tfidf_matrix[doc_index, word_id] = score

tfidf_df = pd.DataFrame(tfidf_matrix, columns=[dictionary[i] for i in range(num_terms)])


In [ ]:
tfidf_df

In [ ]:
X = tfidf_matrix
y = final_df['type'].values

In [ ]:
print((y == 0).sum())
print((y == 1).sum())
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
model_svm = SVC(kernel='rbf', C=4, probability=True)
model_svm.fit(X_train, y_train)

In [ ]:
y_pred_svm = model_svm.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred_svm)
cnf_matrix

In [ ]:
class_names=['no_bully', 'bully'] # name  of classes
fig, ax = plt.subplots(figsize=(3,3))

# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu", fmt='g', cbar=False)
ax.set_xticklabels(class_names, rotation=45, ha='right')  # Rotate x-ticks for readability
ax.set_yticklabels(class_names, rotation=0)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label(svm)')

In [ ]:
target_names = ['no_bully', 'bully']
print(classification_report(y_test, y_pred_svm, target_names=target_names))

In [ ]:
y_score = model_svm.predict_proba(X_test)

In [ ]:
yy=np.eye(2)[y_test]

# Plot ROC curve for each class
plt.figure(figsize=(5,5))

for i in range(2):
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve(yy[:,i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

# Plot the diagonal line for random guessing
plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')

# Label the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='best')
plt.show()

In [ ]:
TP = cnf_matrix[0, 0]
FN = cnf_matrix[0, 1]
FP = cnf_matrix[1, 0]
TN = cnf_matrix[1, 1]

recall = TP / (TP + FN)

precision = TP / (TP + FP)

f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

In [ ]:
# for c=4, poly kerneel accuracy 0.53 & rbf kernel accuracy 0.78
# for c=3  poly kernel accuracy  & rbf kernel accuracy 0.78
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Model Accuracy: {accuracy_svm:.2f}")

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
class_names=['no_bully', 'bully'] # name  of classes
fig, ax = plt.subplots(figsize=(3,3))

# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu", fmt='g', cbar=False)
ax.set_xticklabels(class_names, rotation=45, ha='right')  # Rotate x-ticks for readability
ax.set_yticklabels(class_names, rotation=0)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label(logistic)')


In [ ]:
target_names = ['no_bully', 'bully']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
TP = cnf_matrix[0, 0]
FN = cnf_matrix[0, 1]
FP = cnf_matrix[1, 0]
TN = cnf_matrix[1, 1]

recall = TP / (TP + FN)

precision = TP / (TP + FP)

f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

In [ ]:
y_score = logreg.predict_proba(X_test)

In [ ]:
plt.figure(figsize=(5,5))
y_test_one_hot = label_binarize(y_test, classes=[0, 1,2,3,4])

for i in range(2):
    fpr, tpr, _ = roc_curve(y_test_one_hot[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='best')
plt.show()

In [ ]:
logacc = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {logacc:.2f}")

In [ ]:
booster = GradientBoostingClassifier(loss='log_loss', n_estimators=100, learning_rate=0.1, max_depth=3)
booster.fit(X_train, y_train)

In [ ]:
y_pred = booster.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
class_names=['no_bully', 'bully'] # name  of classes
fig, ax = plt.subplots(figsize=(3,3))

# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu", fmt='g', cbar=False)
ax.set_xticklabels(class_names, rotation=45, ha='right')  # Rotate x-ticks for readability
ax.set_yticklabels(class_names, rotation=0)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label(grad boosting)')


In [ ]:
target_names = ['no_bully', 'bully']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
TP = cnf_matrix[0, 0]
FN = cnf_matrix[0, 1]
FP = cnf_matrix[1, 0]
TN = cnf_matrix[1, 1]

recall = TP / (TP + FN)

precision = TP / (TP + FP)

f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

In [ ]:
y_score = booster.predict_proba(X_test)

In [ ]:
y_score

In [ ]:
y_test_one_hot = label_binarize(y_test, classes=[0,1,2,3,4])
y_test_one_hot

In [ ]:
plt.figure(figsize=(5,5))
y_test_one_hot = label_binarize(y_test, classes=[0, 1,2,3,4])

for i in range(2):
    fpr, tpr, _ = roc_curve(y_test_one_hot[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='best')
plt.show()

In [ ]:
boostacc = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {boostacc:.2f}")